# Qiskit Runtime REST API query & job submission

* https://us-east.quantum-computing.cloud.ibm.com/openapi/ 
* General documentation at https://cloud.ibm.com/apidocs/quantum-computing 
* Users needs to [create an account through IBM Cloud Qiskit Runtime Service and access the API key and Cloud Resource Name (CRN)](https://cloud.ibm.com/apidocs/quantum-computing#authentication). 

## Authenticate via IBM Cloud API key and Cloud Resource Name

In [1]:
with open('cloud_token') as file:
    token=file.read()
with open('crn_service') as file:
    crn_service = file.read()

In [2]:
import requests

url = 'https://iam.cloud.ibm.com/identity/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
data='grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey='+token
auth_response = requests.post(url, headers= headers, data=data)
auth_id=auth_response.json()['access_token']
auth_type = auth_response.json()['token_type']

## GET available backends

In [3]:
url_backends = 'https://us-east.quantum-computing.cloud.ibm.com/backends'
headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id}

backends_response = requests.get(url_backends, headers=headers)
backends_response.json()

{'devices': ['ibm_brisbane',
  'ibm_kyoto',
  'ibm_osaka',
  'ibmq_qasm_simulator',
  'simulator_stabilizer',
  'simulator_mps',
  'simulator_extended_stabilizer',
  'simulator_statevector']}

In [12]:
backend = backends_response.json()['devices'][1]
backend

'ibm_kyoto'

## Run a job

### set up qasm circuit

In [13]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

Note: From March 1, 2024, Qiskit Runtime will require that circuits and observables are transformed to use only ISA (Instruction Set Architecture) instructions supported by the system before being submitted to the primitives.
This change also streamlines service operations to produce faster results and make more efficient use of our fleet of quantum systems. For this reason, establishing the backend will no longer be optional and will become mandatory. See the [transpilation documentation](https://docs.quantum-computing.ibm.com/transpile) for instructions to transform circuits.
* Instances using Q-CTRL performance management do not need to transform circuits or observables.

### run single job via API

In [14]:
import requests

url = 'https://us-east.quantum-computing.cloud.ibm.com/jobs'

headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id,
            'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
            }

job_input = {
    'program_id': 'sampler',
    "backend": backend, 
    "params": {
        "circuits": [qasm_string]#*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code, response.text}")

Job created: {"id":"cnq3nirii28glmfuqst0","backend":"ibm_kyoto"}


## (optional) Create Session 

c.f. documentation at https://cloud.ibm.com/apidocs/quantum-computing 

Note: After March 31, 2024 Qiskit Runtime sessions creation will gain exclusive access to quantum systems, and will be charged for all time from the first job in the session, until the session is closed. Please update your code as soon as possible before that date to avoid unwanted behavior. If you use qiskit-ibm-runtime, update to version 0.20.0 or higher. If you use qiskit-ibm-provider, update to version 0.10.0 or higher. In case you are using the API directly, keep in mind that /jobs will not start a session automatically, so you will have to use the new /sessions endpoint instead.

In [15]:
import json

sessionsUrl = "https://us-east.quantum-computing.cloud.ibm.com/sessions"
 
headersList = {
  "Accept": "application/json",
  'Service-CRN': crn_service,
   'Authorization':auth_type + ' ' + auth_id,
}
 
payload = json.dumps({
  "backend": backend
})
 
response = requests.request("POST", sessionsUrl, data=payload,  headers=headersList)
 
sessionId = response.json()['id']
 
print(response.json())

{'id': '87654617-2ea5-4caf-ace5-76d1337ae0c9', 'backend_name': '', 'interactive_ttl': 60, 'max_ttl': 28800, 'active_ttl': 28800, 'state': 'open', 'accepting_jobs': True, 'mode': None}


### Run jobs in the created Session

In [17]:
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "session_id": sessionId,
    "params": {
        "circuits": [qasm_string]*2
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cnq3odrii28glmfuqt7g","backend":"ibm_kyoto","session_id":"87654617-2ea5-4caf-ace5-76d1337ae0c9"}


### Query Websocket for live job update and close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. So we will listen for a status change of the last job via websocket and close the Session when done.

In [20]:
import asyncio
import websockets

async def receive_status_updates(api_key):
    uri = "wss://us-east.quantum-computing.cloud.ibm.com/stream/jobs/"+job_id 
    extra_headers = {'Authorization': f'Bearer {api_key}',
                     'Service-CRN': crn_service}

    async with websockets.connect(uri, extra_headers=extra_headers) as websocket:
        while True:
            try:
                message = await websocket.recv()
                print(f"Received message: {message}")  
            except websockets.exceptions.ConnectionClosed:
                print("WebSocket connection closed.")
                break

await receive_status_updates(auth_id)

#subsequently closing Session
status_result = requests.get(url+'/'+job_id,headers=headers)
sessionsurl = 'https://runtime-us-east.quantum-computing.ibm.com/sessions/'+sessionId+'/close'

if status_result.json()['state']['status']=='Completed':
   response = requests.delete(sessionsurl,headers=headers)
else:
   print('waiting for job to complete before closing Session '+sessionId)

if response.status_code==204:
    print('Session '+sessionId+' was closed')

Received message: Listening to results for job: cnq3odrii28glmfuqt7g

Received message: {"quasi_dists": [{"0": 0.11901811986990868, "1": 0.8809818801300914}, {"0": 0.11950209075422027, "1": 0.8804979092457796}], "metadata": [{"shots": 4000, "circuit_metadata": {}, "readout_mitigation_overhead": 3.941980554027381, "readout_mitigation_time": 0.0735075231641531}, {"shots": 4000, "circuit_metadata": {}, "readout_mitigation_overhead": 3.941980554027381, "readout_mitigation_time": 0.1048154029995203}]}
WebSocket connection closed.


### Get results

In [21]:
response_status= requests.get(url+'/'+job_id, headers=headers)
response_status.json().get('state')

{'status': 'Completed'}

In [22]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'0': 0.11901811986990868, '1': 0.8809818801300914},
  {'0': 0.11950209075422027, '1': 0.8804979092457796}],
 'metadata': [{'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 3.941980554027381,
   'readout_mitigation_time': 0.0735075231641531},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 3.941980554027381,
   'readout_mitigation_time': 0.1048154029995203}]}